#### DataNatureで使うテキストファイル（タブ区切り）をDataFrameとして処理を行い、csvファイルとして出力する。
#### 在庫ファイル用

In [ ]:
# google drive mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# google driveのpathとinput、outputのファイル名を指定
input_path = "/content/drive/MyDrive/Purchased_products/txt/"
output_path = "/content/drive/MyDrive/Purchased_products/csv/"
input_file_name = "Z2009_2022.txt"
output_file_name = "I_Z_2009_2022.csv"

# 出力ファイルの項目を指定
columns_name = ""

In [ ]:
import numpy as np
import pandas as pd
import re

In [ ]:
# テキストファイル読込
df = pd.read_table(input_path + input_file_name, encoding="cp932")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


### 半角￥マークつきの金額部分の数値のみ取り出す
###### https://torisky.com/python%EF%BC%9A%E5%8D%8A%E8%A7%92%EF%BF%A5%E3%83%9E%E3%83%BC%E3%82%AF%E3%81%A4%E3%81%8D%E3%81%AE%E9%87%91%E9%A1%8D%E9%83%A8%E5%88%86%E3%81%AE%E6%95%B0%E5%80%A4%E3%81%AE%E3%81%BF%E5%8F%96%E3%82%8A/

In [ ]:
df = df.fillna(0)

In [ ]:
def mark_num_change(mark):
    """
        半角￥マークつきの金額部分の数値のみ取り出す
              例）販売価格 \824,529 --> 824529
    """
    if len(mark) >= 2:
        if mark[-2] == ".":
            mark = mark[:-2]
    # 「\」と「,」と数字以外の文字を消去する
    after = re.sub(r"\D", "", mark)
    # マイナス対応
    if "-" in mark:
        after = int(after) * (-1)
    else:
        after = int(after)
    return int(after)

In [ ]:
# 列名を退避
columns = df.columns

In [ ]:
columns

Index(['ロケーションNo', 'ロケーション名', '棚卸月', '品目', '品名', '規格', '大分類CD', '大分類名', '小分類',
       '勘定科目CD', '勘定科目名', '予備1', '予備2', '在庫数', '在庫単価', '在庫金額'],
      dtype='object')

In [ ]:
# nan対応
df = df.fillna(0)
# イレギュラー
df = df[df["品目"] != "-"]

In [ ]:
# DataFrameをnp.arrayに変換　処理スピードを上げるため
df = np.array(df)

In [ ]:
# 在庫数、在庫単価、在庫金額から￥などの文字を除いて数値にする
columns = list(columns)
a = columns.index("在庫数")
b = columns.index("在庫単価")
c = columns.index("在庫金額")
d = columns.index("品目")

for i in range(len(df)):
    df[i, a] = mark_num_change(str(df[i,a]))
    df[i, b] = mark_num_change(str(df[i,b]))
    df[i, c] = mark_num_change(str(df[i,c]))
    if output_file_name[0] != "I":
            df[i, d] = mark_num_change(str(df[i,d]))

In [ ]:
# DataFrameに戻す
df = pd.DataFrame(df, columns=columns)

In [ ]:
# タイプをIntに
if output_file_name[0] != "I":
    df["品目"] = df["品目"].astype(int)    
df["在庫数"] = df["在庫数"].astype(int)
df["在庫単価"] = df["在庫単価"].astype(int)
df["在庫金額"] = df["在庫金額"].astype(int)

In [ ]:
# ファイル出力
df.to_csv(output_path + output_file_name,  encoding="utf_8_sig")